# Predicting House Sale Prices

In this project, we'll be working with housing data for the city of Ames, Iowa from 2006 to 2010.  Information on the different columns contained in the dataset can be found [here](https://s3.amazonaws.com/dq-content/307/data_description.txt).

We'll be building a linear regression model, fitting it, and cleaning the data/selecting features in an effort to create house sale price predictions.

We will start by importing the data and setting up a pipeline of functions that will let us quickly iterate on different models.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

In [2]:
housing_data = pd.read_csv("AmesHousing.tsv", delimiter="\t")
housing_data.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [3]:
def transform_features(df):
    return df

def select_features(df):
    return df[["Gr Liv Area", "SalePrice"]]

def train_and_test(df):
    train = df[:1460]
    test = df[1460:]
    
    # Select just the numeric datatype columns to use in fitting our model
    train_numeric = train.select_dtypes(include=["integer", "float"])
    test_numeric = test.select_dtypes(include=["integer", "float"])
    
    # Remove SalePrice from our training/test features, given that it's our target column
    features = train_numeric.columns.drop("SalePrice")
    
    # Train the model using feature columns and generate SalePrice predictions
    lr = LinearRegression()
    lr.fit(train_numeric[features], train_numeric["SalePrice"])
    predictions = lr.predict(test_numeric[features])
    
    # Calculate RMSE value comparing predictions with existing SalePrice values in test data
    rmse = np.sqrt(mean_squared_error(test_numeric["SalePrice"], predictions))
    
    return rmse

transformed_data = transform_features(housing_data)
filtered_data = select_features(transformed_data)
data_rmse = train_and_test(filtered_data)

data_rmse

57088.25161263909

## Feature Engineering

Now we'll need to handle missing values.

For all columns:
- Drop any column with 5% or more missing values (for now)

For text columns:
- Drop any column with 1 more more missing values (for now)

For numerical columns:
- Fill missing values with the most common value in that column

#### 1. Drop columns with 5% or more missing values

In [4]:
# Create Series object reflecting # of null values in each column
num_missing = housing_data.isnull().sum()

In [5]:
# Filter to columns that contain >5% missing values
cols_to_drop = num_missing[(num_missing > len(housing_data) / 20)]
cols_to_drop

Lot Frontage      490
Alley            2732
Fireplace Qu     1422
Garage Type       157
Garage Yr Blt     159
Garage Finish     159
Garage Qual       159
Garage Cond       159
Pool QC          2917
Fence            2358
Misc Feature     2824
dtype: int64

In [6]:
# Remove these columns from the dataset
housing_data = housing_data.drop(cols_to_drop.index, axis=1)

#### 2. Drop text columns with 1 or more missing values

In [7]:
# Filter to text columns with 1+ missing values
text_missing_counts = housing_data.select_dtypes(include=["object"]).isnull().sum().sort_values(ascending=False)
text_cols_to_drop = text_missing_counts[text_missing_counts > 0]
text_cols_to_drop

Bsmt Exposure     83
BsmtFin Type 2    81
BsmtFin Type 1    80
Bsmt Qual         80
Bsmt Cond         80
Mas Vnr Type      23
Electrical         1
dtype: int64

In [8]:
# Drop these columns from the dataset
housing_data = housing_data.drop(text_cols_to_drop.index, axis=1)

#### 3. Remaining numerical columns: fill in missing values with most common value in each column

In [9]:
# Filter to remaining columns with numerical datatypes, which have missing values
num_cols_missing_counts = housing_data.select_dtypes(include = ["float", "integer"]).isnull().sum().sort_values(ascending=False)
fixable_num_cols = num_cols_missing_counts[num_cols_missing_counts > 0]
fixable_num_cols

Mas Vnr Area      23
Bsmt Half Bath     2
Bsmt Full Bath     2
Garage Cars        1
BsmtFin SF 1       1
Total Bsmt SF      1
Bsmt Unf SF        1
BsmtFin SF 2       1
Garage Area        1
dtype: int64

In [10]:
# Determine most common value for each column
replacement_vals = housing_data[fixable_num_cols.index].mode().to_dict(orient="records")[0]
replacement_vals

{'Mas Vnr Area': 0.0,
 'Bsmt Half Bath': 0.0,
 'Bsmt Full Bath': 0.0,
 'Garage Cars': 2.0,
 'BsmtFin SF 1': 0.0,
 'Total Bsmt SF': 0.0,
 'Bsmt Unf SF': 0.0,
 'BsmtFin SF 2': 0.0,
 'Garage Area': 0.0}

In [11]:
# Fill missing values in with mode values above
housing_data = housing_data.fillna(replacement_vals)

In [12]:
# Ensure there are no missing values remaining in the dataset
housing_data.isnull().sum().value_counts()

0    64
dtype: int64

Next, we can use a few existing columns to create new features that better capture information that will be useful for our model.

A couple of good options would be `"Years Before Sale"` (representing how long a house went after being built before it was sold) and `"Years After Remod"` (representing how many years it was, after a house was remodeled or added on to, before it was sold.

In [13]:
years_sold = housing_data["Yr Sold"] - housing_data["Year Built"]

# Check for any negative values, which wouldn't be useful
years_sold[years_sold < 0]

2180   -1
dtype: int64

In [14]:
years_after_remod = housing_data["Yr Sold"] - housing_data["Year Remod/Add"]

# CHeck for any negative values, which wouldn't be useful
years_after_remod[years_after_remod < 0]

1702   -1
2180   -2
2181   -1
dtype: int64

In [15]:
# Establish new columns for new features in dataset
housing_data["Years Before Sale"] = years_sold
housing_data["Years After Remod/Add"] = years_after_remod

# Remove rows with negative values for these new features
housing_data = housing_data.drop([1702, 2180, 2181], axis=0)

# Remove original year columns, which are no longer needed
housing__data = housing_data.drop(["Year Built", "Year Remod/Add"], axis=1)

Now, we'll move on to removing any additional columns that should be dropped:

- Columns not useful for ML
- Columns that leak data about the final sale

In [16]:
# Drop columns not useful for ML (entry ID and order values)
housing_data = housing_data.drop(["Order", "PID"], axis=1)

# Drop columns that leak data about final sale
housing_data = housing_data.drop(["Yr Sold", "Mo Sold", "Sale Condition", "Sale Type"], axis=1)

Now we can update our `transform_features` function to carry out all of the dataset modifications we made above.  We'll re-write all 3 of our functions together.

In [17]:
def transform_features(df):
    num_missing = df.isnull().sum()
    cols_to_drop = num_missing[(num_missing > len(df) / 20)].sort_values()
    df = df.drop(cols_to_drop.index, axis=1)
    
    text_missing_counts = df.select_dtypes(include=["object"]).isnull().sum().sort_values(ascending=False)
    text_cols_to_drop = text_missing_counts[text_missing_counts > 0]
    df = df.drop(text_cols_to_drop.index, axis=1)
    
    num_cols_missing_counts = df.select_dtypes(include = ["float", "int"]).isnull().sum().sort_values(ascending=False)
    fixable_num_cols = num_cols_missing_counts[num_cols_missing_counts > 0]
    replacement_vals = df[fixable_num_cols.index].mode().to_dict(orient="records")[0]
    df = df.fillna(replacement_vals)
    
    years_sold = df["Yr Sold"] - df["Year Built"]
    years_after_remod = df["Yr Sold"] - df["Year Remod/Add"]
    df["Years Before Sale"] = years_sold
    df["Years After Remod/Add"] = years_after_remod
    df = df.drop([1702, 2180, 2181], axis=0)
    df = df.drop(["Year Built", "Year Remod/Add"], axis=1)
    
    df = df.drop(["Order", "PID", "Yr Sold", "Mo Sold", "Sale Condition", "Sale Type"], axis=1)
    
    return df

def select_features(df):
    return df[["Gr Liv Area", "SalePrice"]]

def train_and_test(df):
    train = df[:1460]
    test = df[1460:]
    
    # Select just the numeric datatype columns to use in fitting our model
    train_numeric = train.select_dtypes(include=["integer", "float"])
    test_numeric = test.select_dtypes(include=["integer", "float"])
    
    # Remove SalePrice from our training/test features, given that it's our target column
    features = train_numeric.columns.drop("SalePrice")
    
    # Train the model using feature columns and generate SalePrice predictions
    lr = LinearRegression()
    lr.fit(train_numeric[features], train_numeric["SalePrice"])
    predictions = lr.predict(test_numeric[features])
    
    # Calculate RMSE value comparing predictions with existing SalePrice values in test data
    rmse = np.sqrt(mean_squared_error(test_numeric["SalePrice"], predictions))
    
    return rmse

# Re-read in housing data from original CSV and run functions
housing_df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transformed_data = transform_features(housing_df)
filtered_data = select_features(transformed_data)
data_rmse = train_and_test(filtered_data)

data_rmse

55275.367312413066

## Feature Selection

Now that we've cleaned and transformed a lot of the features in the dataset, it's time to move on to feature selection for numerical features.

In [18]:
num_columns = transformed_data.select_dtypes(include=["float", "integer"])
num_columns.head(5)

,MS SubClass,Lot Area,Overall Qual,Overall Cond,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,SalePrice,Years Before Sale,Years After Remod/Add
0,20,31770,6,5,112.0,639.0,0.0,441.0,1080.0,1656,...,210,62,0,0,0,0,0,215000,50,50
1,20,11622,5,6,0.0,468.0,144.0,270.0,882.0,896,...,140,0,0,0,120,0,0,105000,49,49
2,20,14267,6,6,108.0,923.0,0.0,406.0,1329.0,1329,...,393,36,0,0,0,0,12500,172000,52,52
3,20,11160,7,5,0.0,1065.0,0.0,1045.0,2110.0,2110,...,0,0,0,0,0,0,0,244000,42,42
4,60,13830,5,5,0.0,791.0,0.0,137.0,928.0,928,...,212,34,0,0,0,0,0,189900,13,12


Let's take a look at the correlation coeffiecients (absolute values) for each feature column with `SalePrice`.

In [19]:
abs_corr_coeffs = num_columns.corr()["SalePrice"].abs().sort_values()
abs_corr_coeffs

BsmtFin SF 2             0.006127
Misc Val                 0.019273
3Ssn Porch               0.032268
Bsmt Half Bath           0.035875
Low Qual Fin SF          0.037629
Pool Area                0.068438
MS SubClass              0.085128
Overall Cond             0.101540
Screen Porch             0.112280
Kitchen AbvGr            0.119760
Enclosed Porch           0.128685
Bedroom AbvGr            0.143916
Bsmt Unf SF              0.182751
Lot Area                 0.267520
2nd Flr SF               0.269601
Bsmt Full Bath           0.276258
Half Bath                0.284871
Open Porch SF            0.316262
Wood Deck SF             0.328183
BsmtFin SF 1             0.439284
Fireplaces               0.474831
TotRms AbvGrd            0.498574
Mas Vnr Area             0.506983
Years After Remod/Add    0.534985
Full Bath                0.546118
Years Before Sale        0.558979
1st Flr SF               0.635185
Garage Area              0.641425
Total Bsmt SF            0.644012
Garage Cars   

For now, we'll keep only the columns with coefficient values greater than 0.4 (an admittedly arbitrary choice).  Because we have a pipeline in place, it's easy to adjust/experiment with this value later and see which features result in a better cross validation score in the end.

In [20]:
abs_corr_coeffs[abs_corr_coeffs > 0.4]

BsmtFin SF 1             0.439284
Fireplaces               0.474831
TotRms AbvGrd            0.498574
Mas Vnr Area             0.506983
Years After Remod/Add    0.534985
Full Bath                0.546118
Years Before Sale        0.558979
1st Flr SF               0.635185
Garage Area              0.641425
Total Bsmt SF            0.644012
Garage Cars              0.648361
Gr Liv Area              0.717596
Overall Qual             0.801206
SalePrice                1.000000
Name: SalePrice, dtype: float64

In [21]:
# Drop columns with less than 0.4 correlation with SalePrice
transformed_data = transformed_data.drop(abs_corr_coeffs[abs_corr_coeffs < 0.4].index, axis=1)

### Categorical Variables

Now we can take a look at which columns in our dataset should be converted to the `categorical` datatype.  We can start by creating a list of nominal variables, from the data, that are *meant* to be categorical:

In [22]:
nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]

A couple of questions we should now ask re: converting to categorical datatype:

- If a categorical column has hundreds of unique values/categories, should we keep it?  When we dummy code this column, hundreds of new columns will need to be created in the dataframe.
- Which columns are currently numerical but need to be encoded as categorical instead (because the numbers don't have any semantic meaning)?

First, let's check: of the columns we're still working with at this point, which are included in the above nominal variable list?

In [23]:
transform_cat_cols = []
for col in nominal_features:
    if col in transformed_data.columns:
        transform_cat_cols.append(col)

Now we'll apply a cutoff of 10 unique categories per column (we can experiment with this threshold further, if desired).

In [24]:
# Establish counts of unique categories for selected columns
num_cat_counts = transformed_data[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()

# Drop columns with more than 10 unique categories
transformed_data = transformed_data.drop(num_cat_counts[num_cat_counts > 10].index, axis=1)

Then, we can convert the remaining text columns to category columns, and create our dummy columns.

In [25]:
# Select remaining text columns and convert to category
text_cols = transformed_data.select_dtypes(include=["object"])
for col in text_cols:
    transformed_data[col] = transformed_data[col].astype("category")
    
# Create dummy columns and add back to dataframe
transformed_data = pd.concat([
    transformed_data,
    pd.get_dummies(transformed_data.select_dtypes(include=["category"]))
], axis=1)

# Finally, drop original text/category columns
transformed_data = transformed_data.drop(text_cols, axis=1)

Once again, we can now update our functions; this time, we'll update `select_features`.

In [29]:
def transform_features(df):
    num_missing = df.isnull().sum()
    cols_to_drop = num_missing[(num_missing > len(df) / 20)].sort_values()
    df = df.drop(cols_to_drop.index, axis=1)
    
    text_missing_counts = df.select_dtypes(include=["object"]).isnull().sum().sort_values(ascending=False)
    text_cols_to_drop = text_missing_counts[text_missing_counts > 0]
    df = df.drop(text_cols_to_drop.index, axis=1)
    
    num_cols_missing_counts = df.select_dtypes(include = ["float", "int"]).isnull().sum().sort_values(ascending=False)
    fixable_num_cols = num_cols_missing_counts[num_cols_missing_counts > 0]
    replacement_vals = df[fixable_num_cols.index].mode().to_dict(orient="records")[0]
    df = df.fillna(replacement_vals)
    
    years_sold = df["Yr Sold"] - df["Year Built"]
    years_after_remod = df["Yr Sold"] - df["Year Remod/Add"]
    df["Years Before Sale"] = years_sold
    df["Years After Remod/Add"] = years_after_remod
    df = df.drop([1702, 2180, 2181], axis=0)
    df = df.drop(["Year Built", "Year Remod/Add"], axis=1)
    
    df = df.drop(["Order", "PID", "Yr Sold", "Mo Sold", "Sale Condition", "Sale Type"], axis=1)
    
    return df

def select_features(df, coeff_threshold=0.4, cat_threshold=10):
    numerical_df = df.select_dtypes(include=['integer', 'float'])
    abs_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values()
    df = df.drop(abs_corr_coeffs[abs_corr_coeffs < coeff_threshold].index, axis=1)
    
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]
    
    transform_cat_cols = []
    for col in nominal_features:
        if col in df.columns:
            transform_cat_cols.append(col)

    num_cat_counts = df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values() 
    df = df.drop(num_cat_counts[num_cat_counts > cat_threshold].index, axis=1)
    
    text_cols = df.select_dtypes(include=['object'])
    for col in text_cols:
        df[col] = df[col].astype('category')
    df = pd.concat([df, pd.get_dummies(df.select_dtypes(include=['category']))], axis=1).drop(text_cols,axis=1)
    
    return df

def train_and_test(df):
    train = df[:1460]
    test = df[1460:]
    
    # Select just the numeric datatype columns to use in fitting our model
    train_numeric = train.select_dtypes(include=["integer", "float"])
    test_numeric = test.select_dtypes(include=["integer", "float"])
    
    # Remove SalePrice from our training/test features, given that it's our target column
    features = train_numeric.columns.drop("SalePrice")
    
    # Train the model using feature columns and generate SalePrice predictions
    lr = LinearRegression()
    lr.fit(train_numeric[features], train_numeric["SalePrice"])
    predictions = lr.predict(test_numeric[features])
    
    # Calculate RMSE value comparing predictions with existing SalePrice values in test data
    rmse = np.sqrt(mean_squared_error(test_numeric["SalePrice"], predictions))
    
    return rmse

# Re-read in housing data from original CSV and run functions
housing_df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transformed_data = transform_features(housing_df)
filtered_data = select_features(transformed_data)
data_rmse = train_and_test(filtered_data)

data_rmse

33367.28718340374

## Train and Test

Now for the final part of the pipeline, training and testing. When iterating on different features, using simple validation is a good idea. Let's add a parameter named k to our `train_and_test` function that controls the type of cross validation that occurs.

In [34]:
def train_and_test(df, k=0):
    numeric_df = df.select_dtypes(include=['integer', 'float'])
    features = numeric_df.columns.drop("SalePrice")
    lr = LinearRegression()
    
    # If k is 0, perform holdout validation
    if k == 0:
        
        train = df[:1460]
        test = df[1460:]

        lr.fit(train[features], train["SalePrice"])
        predictions = lr.predict(test[features])
        mse = mean_squared_error(test["SalePrice"], predictions)
        rmse = np.sqrt(mse)

        return rmse
    
    # If k is 1, perform simple cross validation
    if k == 1:
        
        # Randomize all rows using (frac=1) from "df" and return
        shuffled_df = df.sample(frac=1, )
        train = df[:1460]
        test = df[1460:]
        
        lr.fit(train[features], train["SalePrice"])
        predictions_one = lr.predict(test[features])        
        
        mse_one = mean_squared_error(test["SalePrice"], predictions_one)
        rmse_one = np.sqrt(mse_one)
        
        lr.fit(test[features], test["SalePrice"])
        predictions_two = lr.predict(train[features])        
       
        mse_two = mean_squared_error(train["SalePrice"], predictions_two)
        rmse_two = np.sqrt(mse_two)
        
        avg_rmse = np.mean([rmse_one, rmse_two])
        print(rmse_one)
        print(rmse_two)
        return avg_rmse
    
    
    # If k > 1, use KFold to perform k-fold cross validation
    else:
        
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        for train_index, test_index, in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            lr.fit(train[features], train["SalePrice"])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test["SalePrice"], predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        return avg_rmse
        
rmse = train_and_test(filtered_data, k=4)

rmse        

[25492.466794178024, 26300.590474204346, 36054.447982438134, 29105.211426713682]


29238.179169383548

## Next Steps

A couple of potential next steps:

- Continuing iteration on feature engineering: researching additional approaches to feature engineering online for housing data
- Improving feature selection: researching better ways of doing feature selection with categorical columns